In [1]:
import numpy as np

import xrb
from xrb.src.core import *
set_data_path("../../data")
from xrb.binary import load_sse, binary_evolve
import xrb.src.constants as c
from xrb.pop_synth import pop_synth
from xrb.SF_history import sf_history

from astropy.coordinates import SkyCoord

%matplotlib inline

In [2]:
load_sse.load_sse()
sf_history.load_sf_history()

In [3]:
# sf_history.smc_coor = None
# sf_history.load_sf_history()

print len(sf_history.smc_sfh)
print len(sf_history.smc_coor)
print len(sf_history.lmc_sfh)
print len(sf_history.lmc_coor)

460
460
1376
1376


In [4]:
N = 100


t_min, t_max = 5.0, 6.0
N_times = (t_max-t_min) * 10

for t_b in np.arange(N_times)/10.0+t_min:
    
    HMXB, init_params = pop_synth.create_HMXBs(t_b, N_sys=N)
    
    print t_b, len(HMXB)

5.0 0
5.1 0
5.2 0
5.3 0
5.4 0
5.5 0
5.6 0
5.7 0
5.8 0
5.9 0


In [6]:
# Set J0045 parameters
coor_J0045 = SkyCoord('00h45m35.26s', '-73d19m03.32s')

ra_J0045 = coor_J0045.ra.degree
dec_J0045 = coor_J0045.dec.degree
M2_d_J0045 = 8.8  # M2 in Msun
M2_d_J0045_err = 1.8
P_orb_J0045 = 51.17  # P_orb in days
P_orb_J0045_err = 1.0
ecc_J0045 = 0.808  # eccentricity
ecc_J0045_err = 0.05

t_min, t_max = 5.0, 60.0
N_times = (t_max-t_min) * 10
N = 10000

N_good = 0
N_good_J0045_3 = 0
N_good_J0045_2 = 0
for t_b in np.arange(N_times)/10.0+t_min:

    if t_b%1 == 0: print t_b
    
    HMXB, init_params = pop_synth.create_HMXBs(t_b, N_sys=N)
    M1 = init_params['M1']
    ecc = HMXB['ecc']
    
    
    theta_from_J0045 = sf_history.get_theta_proj_degree(ra_J0045, dec_J0045, init_params['ra_b'], init_params['dec_b'])
        
    fwd_M_NS, fwd_M2, fwd_L_x, fwd_v_sys, fwd_M2_dot, fwd_A, fwd_ecc, fwd_theta, fwd_k = \
        pop_synth.full_forward(init_params['M1'], init_params['M2'], init_params['A'], \
                 init_params['ecc'], init_params['v_k'], init_params['theta'], \
                 init_params['phi'], t_b)
    
    v_sys = fwd_v_sys
    
    P_orb = binary_evolve.A_to_P(fwd_M_NS, fwd_M2, fwd_A)
    
    
    theta_max = c.rad_to_deg * (t_b - load_sse.func_sse_tmax(M1)) * v_sys / c.dist_SMC * c.yr_to_sec * 1.0e6


    # Now, select the systems similar to J0045
    for i in np.arange(len(HMXB)):
        if theta_from_J0045[i] < theta_max[i] and \
           np.abs(M2_d_J0045 - fwd_M2[i]) < 3.0*M2_d_J0045_err and \
           np.abs(P_orb_J0045 - P_orb[i]) < 3.0*P_orb_J0045_err and \
           np.abs(ecc_J0045 - ecc[i]) < 3.0*ecc_J0045_err:
        
            N_good_J0045_3 = N_good_J0045_3 + 1
            
        if theta_from_J0045[i] < theta_max[i] and \
           np.abs(M2_d_J0045 - fwd_M2[i]) < 2.0*M2_d_J0045_err and \
           np.abs(P_orb_J0045 - P_orb[i]) < 2.0*P_orb_J0045_err and \
           np.abs(ecc_J0045 - ecc[i]) < 2.0*ecc_J0045_err:
        
            N_good_J0045_2 = N_good_J0045_2 + 1

    
    N_good = N_good + len(HMXB)

print "Found", N_good, "HMXBs out of", float(N) * float(N_times), "binaries"
print "Found", N_good_J0045_3, "HMXBs similar to J0045 at 3-sigma"
print "Found", N_good_J0045_2, "HMXBs similar to J0045 at 2-sigma"


5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
Found 249228 HMXBs out of 5500000.0 binaries
Found 830 HMXBs similar to J0045 at 3-sigma
Found 249 HMXBs similar to J0045 at 2-sigma
